# Mini Project 5: World Democracy and Freedom

### Learning Objective
Gaining experience in data preparation, unsupervised machine learning analysis, and visualisation of clusters.

### Project Objective
For the last 50 years the organisation Freedom House (https://freedomhouse.org) monitors and protects the democracy values, political rights and civil liberties in the world. We would like to get insights about the status of freedom of all countries in 2023.

## Tasks

### 1. Research the work of Freedom House and choose some of their data collections, relevant to the objective.

In [3]:
#Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
df = pd.read_csv('/Users/emiliocastrolagunas/Desktop/Sem4/BI/Notebooks/Data/freedom.csv')
#df = pd.read_csv('../')

In [13]:
#idea of general looks of data
df.head()

,country,1972,1973,1974,1975,1976,1977,1978,1979,1980,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,4.5,6.5,6.5,6.5,6.5,6.0,7.0,7.0,7.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.5,5.5
1,Albania,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,Algeria,6.0,6.0,6.0,6.5,6.0,6.0,6.0,6.0,6.0,...,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5
3,Andorra,3.5,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,Angola,NaN,NaN,NaN,6.0,6.0,7.0,7.0,7.0,7.0,...,5.5,5.5,5.5,5.5,5.5,5.5,6.0,6.0,6.0,5.5


In [10]:
df.shape

(193, 48)

In [12]:
#checking for null-values
df.isnull().sum()

country      0
1972        51
1973        50
1974        48
1975        42
1976        41
1977        45
1978        42
1979        39
1980        38
1981        36
1982       193
1983        35
1984        34
1985        34
1986        34
1987        34
1988        34
1989        33
1990        33
1991        14
1992        10
1993         5
1994         4
1995         4
1996         4
1997         4
1998         4
1999         3
2000         3
2001         3
2002         3
2003         3
2004         3
2005         3
2006         1
2007         1
2008         1
2009         1
2010         1
2011         0
2012         0
2013         0
2014         0
2015         0
2016         0
2017         0
2018         0
dtype: int64

### 2. Clean and explore the data and prepare it for clustering


### 3. Choose an unsupervised ML algorithm and apply it to create clusters of countries, based on their freedom assessment measure.

### 4. Create a geographical map of the world and colours the countries on it to reflect on their freedom status (cluster).